<a href="https://colab.research.google.com/github/valentinocc/Keras_cifar10/blob/master/keras_rnn_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==1.13.1

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Activation, add, MaxPooling2D, Dense, Flatten, Input
from tensorflow.keras.models import Model

%load_ext tensorboard.notebook

In [0]:
def residual_block(x, filter_amount, project=False):  

  if (project):#project x_input to the new dimensions
    x_input = MaxPooling2D(pool_size = (2, 2), strides = (2, 2))(x)
    x_input = Conv2D(filter_amount, (1, 1), strides = (1, 1), padding = 'same')(x_input)
    x = conv_elu_block(x, filter_amount, strides = (2, 2))
    
  else:
    x_input = x
    x = conv_elu_block(x, filter_amount)
  
  x = conv_elu_block(x, filter_amount)
  
  residual_block = add([x, x_input])
  
  
  return residual_block
  
  
  
def conv_elu_block(x, filter_amount, strides = (1,1)):
  
  x = Conv2D(filter_amount, (3, 3), strides = strides, padding = 'same')(x)
  x = Activation('elu')(x)
  
  return x


def residual_neural_network(image_input, stack_amount, class_amount):
    
  x = Conv2D(16, (3, 3), strides = (1, 1), padding = 'same')(image_input)
  
  for i in range(stack_amount):
    x = residual_block(x, 16)
  
  x = residual_block(x, 32, project=True)
  
  for i in range(stack_amount):
    x = residual_block(x, 32)
  
  x = residual_block(x, 64, project=True)
  
  for i in range(stack_amount):
    x = residual_block(x, 64)
    
  x = Activation('elu')(x)
  x = MaxPooling2D(pool_size = (2, 2), strides = (2, 2))(x)
  x = Flatten()(x)
  x = Dense(class_amount, activation = 'softmax')(x)
  
  return x

def configure_augmented_data_generator(x_train):
  data_generator = ImageDataGenerator(rotation_range = 5, width_shift_range = 0.15, height_shift_range = 0.15, zoom_range = 0.13, horizontal_flip = True)
  data_generator.fit(x_train)
  
  return data_generator

In [6]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.006*8, shuffle= True)

170500096/170498071 [==============================] - 9s 0us/step


In [7]:
INPUT_SHAPE = x_train.shape[1:]
STACK_AMOUNT = 9
CLASS_AMOUNT = 10

image_input = Input(shape=INPUT_SHAPE)
output = residual_neural_network(image_input, STACK_AMOUNT, CLASS_AMOUNT)
ResNet = Model(image_input, output)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
"""TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tpu_model = tf.contrib.tpu.keras_to_tpu_model(ResNet, strategy = tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(tpu = TPU_WORKER)))
tpu_model.compile( optimizer = tf.train.AdamOptimizer(learning_rate = 0.002), loss = tf.keras.losses.sparse_categorical_crossentropy, metrics = ['sparse_categorical_accuracy'])
"""

In [8]:
EPOCHS = 5

data_generator = configure_augmented_data_generator(x_train)
ResNet.fit_generator(data_generator.flow(x_train, y_train, batch_size = 32*8), steps_per_epoch = len(x_train) / (32*8), epochs=EPOCHS, validation_data = (x_valid, y_valid))
ResNet.evaluate(x_test, y_test, batch_size = 32*8)

AttributeError: ignored

In [2]:
from __future__ import print_function
import tensorflow
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Flatten, Input, add, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
import numpy as np
import os
from sklearn.model_selection import train_test_split


batch_size = 32 * 8 # *8 for the 8 TPU cores
epochs = 200
data_augmentation = True
num_classes = 10
subtract_pixel_mean = True

n = 3

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)


def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=(3, 3),
                  strides=(1, 1),
                  padding='same',
                  kernel_initializer='glorot_uniform')
                  #kernel_regularizer=l2(1e-4)

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

def residual_block(x, filter_amount, project=False):  

  if (project):#project x_input to the new dimensions
    x_input = MaxPooling2D(pool_size = (2, 2), strides = (2, 2))(x)
    x_input = Conv2D(filter_amount, (1, 1), strides = (1, 1))(x_input)
    x = conv_elu_block(x, filter_amount, strides = (2, 2))
    
  else:
    x_input = x
    x = conv_elu_block(x, filter_amount)
  
  x = conv_elu_block(x, filter_amount)
  
  residual_block = add([x, x_input])
  
  
  return residual_block
  
  
  
def conv_elu_block(x, filter_amount, strides = (1,1)):
  
  x = Conv2D(filter_amount, (3, 3), strides = strides, padding = 'same')(x)
  x = Activation('elu')(x)
  
  return x
  
def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """

    # Start model definition.
    filter_amount = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = Conv2D(16, (3, 3), strides = (1, 1), padding = 'same')(inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            if stack > 0 and res_block == 0:  # first layer but not first stack
                x = residual_block(x, filter_amount, project = True)
            else:
                x = residual_block(x, filter_amount, project = False)
        filter_amount *= 2
    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = MaxPooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model
  
def configure_augmented_data_generator(x_train):
  data_generator = ImageDataGenerator(rotation_range = 5, width_shift_range = 0.15, height_shift_range = 0.15, zoom_range = 0.13, horizontal_flip = True)
  data_generator.fit(x_train)
  
  return data_generator

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [13]:
print(tensorflow.__version__)

1.14.0-rc1


In [10]:
model = resnet_v1(input_shape=input_shape, depth=depth)

TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tpu_model = tensorflow.contrib.tpu.keras_to_tpu_model(model, strategy = tensorflow.contrib.tpu.TPUDistributionStrategy(tensorflow.contrib.cluster_resolver.TPUClusterResolver(tpu = TPU_WORKER)))

tpu_model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])
tpu_model.summary()
"""
(x_train, y_train), (x_test, y_test) = tensorflow.keras.datasets.cifar10.load_data()
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.006*8, shuffle= True)
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)
"""
EPOCHS = 200

"""data_generator = configure_augmented_data_generator(x_train)
tpu_model.fit_generator(data_generator.flow(x_train, y_train, batch_size = 32*8), epochs=EPOCHS, validation_data = (x_test, y_test))"""
tpu_model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
tpu_model.evaluate(x_test, y_test, batch_size = 32*8)

print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

INFO:tensorflow:Querying Tensorflow master (grpc://10.53.89.202:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 17749368447737351137)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 12912880486433505638)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 8492799238564760701)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 15717615640933417762)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 16464959520372421550)
INFO:tensorflow:*** Available Device: _DeviceAttribute

AttributeError: ignored